In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# If i want to update stardist
import sys
import socket
from pystackreg import StackReg
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from skimage import filters
import numpy as np
from tifffile import imsave as tif_imsave
from tifffile import imwrite as tif_imwrite

from tifffile import imread as tif_imread
from csbdeep.utils import normalize

import matplotlib.pyplot as plt
import sys
from tqdm.auto import tqdm
sys.path.append('/home/oamsalem/code/Projects/EZfish')
sys.path.append('/home/oamsalem/code/Projects/EZfish')
sys.path.append(r'C:\Users\oamsalem\Projects\EZfish')

from sbxreader import sbx_memmap
import os
from pathlib import Path
from skimage.morphology import disk
from funs.par_funcs import regist_stack_parallel,filter_data_parallel
#import regist_stack_parallel

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
def set_reg_obj(reg_method):
    if reg_method =='AFFINE':
        sr = StackReg(StackReg.AFFINE)
    elif reg_method == 'RIGID_BODY':
        sr = StackReg(StackReg.RIGID_BODY)
    elif reg_method == '':
        return None
    else:
        raise Exception("Implement")
    return sr

def regist_stack(stack, filtered_stack, sr):
    if sr==None:
        return stack
    
    reg_stack = np.zeros_like(filtered_stack)
    #raise Exception("I have a bug herer, I am registering all planes per t, this is stupid!!")
    for pn in np.arange(filtered_stack.shape[1]):
        tmats = sr.register_stack(filtered_stack[:,pn].astype(np.float64), reference='mean')
        reg_stack[:,pn] = sr.transform_stack(stack[:,pn].astype(np.float64), tmats=tmats)
    return reg_stack

def filter_data(stack, filter_pipe):
    if filter_pipe == '':
        return stack.astype(np.float64)
    
    filtered_stack = stack.copy().astype(np.float64)

    for p in range(stack.shape[0]):
        for s in range(stack[p].shape[0]):
            img = stack[p][s]
            if 'guas' in filter_pipe:
                img = img - gaussian(img,filter_pipe['guas'])
            if 'med' in filter_pipe:
                img = img -  filters.median(img, disk(filter_pipe['med']))
            filtered_stack[p][s] = img
    return filtered_stack

def apply_func(stack,fun):
    if fun == 'mean':
        stack = np.mean(stack,0)
    if fun == 'max':
        stack = np.max(stack,0)
    if fun == 'median':
        stack = np.median(stack,0)
    return stack.astype(np.float32)

In [4]:
base_path = r'//nasquatch/data/2p/peter/'

paths = ['in_vivo/231108_DG61PL62/DG61PL62_231108_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref85_wxy.npy',
        'in_vivo/231108_DG61PL62/DG61PL62_231108_101_full_wch0_aniso4_scale4AFS2PL4_toref34_wxy.npy']

jumps = [52,
        52]

In [32]:
base_path = r'//nasquatch/data/2p/peter/'

paths = ['in_vivo/231107_DG44PL58/DG44PL58_231107_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref85_wxy.npy',
        'in_vivo/231107_DG44PL58/DG44PL58_231107_101_full_wch0_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/231106_DG52PL68/DG52PL68_231106_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref85_wxy.npy',
        'in_vivo/231106_DG52PL68/DG52PL68_231106_101_full_wch0_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/231105_DG39PL32/DG39PL32_231105_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref85_wxy.npy',
        'in_vivo/231105_DG39PL32/DG39PL32_231105_201_full_wch0_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/231105_DG43PL32/DG43PL32_231105_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref19_wxy.npy',
        'in_vivo/231031_DG53PL28/DG53PL28_231031_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref85_wxy.npy',
        'in_vivo/231031_DG53PL28/DG53PL28_231031_101_full_wch0_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/231030_DG60PL67/DG60PL67_231030_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref49_wxy.npy',
        'in_vivo/231030_DG60PL67/DG60PL67_231030_101_full_wch0_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/231029_DG58PL64/DG58PL64_231029_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref10_wxy.npy',
        'in_vivo/231027_DG57PL69/DG57PL69_231027_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref85_wxy.npy',
        'in_vivo/231025_DG56PL62/DG56PL62_231025_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref85_wxy.npy',
        'in_vivo/231025_DG56PL62/DG56PL62_231025_101_full_wch0_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/231024_DG54PL61/DG54PL61_231024_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/231024_DG54PL61/DG54PL61_231024_101_full_wch0_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/231019_DG53PL68/DG53PL68_231019_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref85_wxy.npy',
        'in_vivo/231018_DG52PL66/DG52PL66_231018_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/231018_DG52PL66/DG52PL66_231018_101_full_wch0_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/231017_DG49PL70/DG49PL70_231017_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/230909_DG88PL58/DG88PL58_230909_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/230909_DG88PL58/DG88PL58_230909_101_full_wch0_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/230908_PK109PL52/PK109PL52_230908_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref65_wxy.npy',
        'in_vivo/230908_PK109PL52/PK109PL52_230908_101_full_wch0_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/230907_PK108PL54/PK108PL54_230907_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref80_wxy.npy',
        'in_vivo/230906_PS57PL56/PS57PL56_230906_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref60_wxy.npy',
        'in_vivo/230906_PS57PL56/PS57PL56_230906_101_full_wch0_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/230905_PS56PL58/PS56PL58_230905_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref89_wxy.npy',
        'in_vivo/230905_PS56PL58/PS56PL58_230905_101_full_wch0_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/230904_PS54PL60/PS54PL60_230904_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref89_wxy.npy',
        'in_vivo/230904_PS54PL60/PS54PL60_230904_101_full_wch0_aniso4_scale4AFS2PL4_toref38_wxy.npy',
        'in_vivo/230902_DG90PL61/DG90PL61_230902_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref75_wxy.npy',
        'in_vivo/230902_DG90PL61/DG90PL61_230902_101_full_wch0_aniso4_scale4AFS2PL4_toref38_wxy.npy',
        'in_vivo/230901_DG89PL32/DG89PL32_230901_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref30_wxy.npy',
        'in_vivo/230901_DG89PL32/DG89PL32_230901_101_full_wch0_aniso4_scale4AFS2PL4_toref38_wxy.npy']

jumps = [52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        31,
        52,
        31,
        52,
        52,
        31,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52]

In [27]:
base_path = r'//nasquatch/data/2p/peter/'

paths = ['slice/20230831_SL1PL55/PL55_230831_001_full_wch0_aniso4_scale4AFS2PL4_toref52_wxy.npy',
        'slice/20230831_SL1PL55/SL1PL55_230831_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref28_wxy.npy',
        'slice/20230830_SL2PL60/SL2PL60_230830_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref28_wxy.npy',
        'slice/20230830_SL1PL28/PL28_230830_001_full_wch0_aniso4_scale4AFS2PL4_toref52_wxy.npy',
        'slice/20230830_SL1PL28/SL1PL28_230830_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref28_wxy.npy',
        'slice/20230829_SL3PL61/SL3PL61_230829_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref28_wxy.npy',
        'slice/20230829_SL2PL39/SL2PL39_230829_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref28_wxy.npy',
        'slice/20230829_SL1PL56/PL56_230829_001_full_wch0_aniso4_scale4AFS2PL4_toref52_wxy.npy',
        'slice/20230829_SL1PL56/SL1PL56_230829_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref28_wxy.npy',
        'slice/20230824_SL2PL54/SL2PL54_230824_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref33_wxy.npy',
        'slice/20230824_SL1PL60/PL60_230824_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        'slice/20230824_SL1PL60/PL60_230824_101_full_wch0_aniso4_scale4AFS2PL4_toref20_wxy.npy',
        'slice/20230824_SL1PL60/SL1PL60_230824_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref22_wxy.npy',
        'slice/20230822_SL3PL58/SL3PL58_230822_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref28_wxy.npy',
        'slice/20230822_SL2PL52/PL52_230822_001_full_wch0_aniso4_scale4AFS2PL4_toref52_wxy.npy',
        'slice/20230822_SL2PL52/SL2PL52_230822_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref28_wxy.npy',
        'slice/20230822_SL1PL32/SL1PL32_230822_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref28_wxy.npy']

jumps = np.repeat(52,len(paths)).tolist()

In [19]:
# second set of 2023 dipping with warp to ch0!
base_path = r'//nasquatch/data/2p/peter/'

paths = ['dipping/20230825_PL57/PL57_230825_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        'dipping/20230825_PL39/PL39_230825_001_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230823_PL61/PL61_230823_001_full_wch0_aniso4_scale4AFS2PL4_toref27_wxy.npy',
        'dipping/20230823_PL48/PL48_230823_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230823_PL28/PL28_230823_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        'dipping/20230823_PL28/PL28_230823_101_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230823_PL28/PL28_230823_201_full_wch0_aniso4_scale4AFS2PL4_toref46_wxy.npy',
        'dipping/20230821_PL57/PL57_230821_001_full_wch0_aniso4_scale4AFS2PL4_toref22_wxy.npy',
        'dipping/20230819_PL62/PL62_230819_001_full_wch0_aniso4_scale4AFS2PL4_toref20_wxy.npy',
        'dipping/20230819_PL43/PL43_230819_001_full_wch0_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'dipping/20230819_PL43/PL43_230819_101_full_wch0_aniso4_scale4AFS2PL4_toref31_wxy.npy',
        'dipping/20230819_PL43/PL43_230819_201_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy']

jumps = np.repeat(52,len(paths)).tolist()

In [14]:
# 2023 dipping - two sets, one has jumps 31 one 52; here use warp to ch0!!
# this if first set 9/15 onward
base_path = r'//nasquatch/data/2p/peter/'

paths = ['dipping/20230927_PL58/PL58_230927_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230927_PL58/PL58_230927_101_full_wch0_aniso4_scale4AFS2PL4_toref31_wxy.npy',
        'dipping/20230927_PL58/PL58_230927_201_full_wch0_aniso4_scale4AFS2PL4_toref30_wxy.npy',
        'dipping/20230927_PL58/PL58_230927_301_full_wch0_aniso4_scale4AFS2PL4_toref30_wxy.npy',
        'dipping/20230927_PL34/PL34_230927_001_full_wch0_aniso4_scale4AFS2PL4_toref20_wxy.npy',
        'dipping/20230927_PL34/PL34_230927_101_full_wch0_aniso4_scale4AFS2PL4_toref35_wxy.npy',
        'dipping/20230927_PL34/PL34_230927_201_full_wch0_aniso4_scale4AFS2PL4_toref37_wxy.npy',
        'dipping/20230927_PL34/PL34_230927_301_full_wch0_aniso4_scale4AFS2PL4_toref30_wxy.npy',
        'dipping/20230926_PL56/PL56_230926_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230926_PL56/PL56_230926_101_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230926_PL56/PL56_230926_201_full_wch0_aniso4_scale4AFS2PL4_toref32_wxy.npy',
        'dipping/20230926_PL56/PL56_230926_301_full_wch0_aniso4_scale4AFS2PL4_toref35_wxy.npy',
        'dipping/20230926_PL56/PL56_230926_401_full_wch0_aniso4_scale4AFS2PL4_toref30_wxy.npy',
        'dipping/20230926_PL56/PL56_230926_501_full_wch0_aniso4_scale4AFS2PL4_toref31_wxy.npy',
        'dipping/20230925_PL58/PL58_230925_001_full_wch0_aniso4_scale4AFS2PL4_toref30_wxy.npy',
        'dipping/20230925_PL58/PL58_230925_101_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230925_PL58/PL58_230925_201_full_wch0_aniso4_scale4AFS2PL4_toref37_wxy.npy',
        'dipping/20230925_PL58/PL58_230925_301_full_wch0_aniso4_scale4AFS2PL4_toref31_wxy.npy',
        'dipping/20230925_PL58/PL58_230925_401_full_wch0_aniso4_scale4AFS2PL4_toref31_wxy.npy',
        'dipping/20230923_PL35/PL35_230923_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230923_PL35/PL35_230923_101_full_wch0_aniso4_scale4AFS2PL4_toref37_wxy.npy',
        'dipping/20230923_PL35/PL35_230923_201_full_wch0_aniso4_scale4AFS2PL4_toref37_wxy.npy',
        'dipping/20230923_PL35/PL35_230923_301_full_wch0_aniso4_scale4AFS2PL4_toref26_wxy.npy',
        'dipping/20230923_PL35/PL35_230923_401_full_wch0_aniso4_scale4AFS2PL4_toref36_wxy.npy',
        'dipping/20230923_PL35/PL35_230923_501_full_wch0_aniso4_scale4AFS2PL4_toref31_wxy.npy',
        'dipping/20230922_PL28/PL28_230922_001_full_wch0_aniso4_scale4AFS2PL4_toref35_wxy.npy',
        'dipping/20230922_PL28/PL28_230922_101_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230922_PL28/PL28_230922_201_full_wch0_aniso4_scale4AFS2PL4_toref19_wxy.npy',
        'dipping/20230922_PL28/PL28_230922_301_full_wch0_aniso4_scale4AFS2PL4_toref35_wxy.npy',
        'dipping/20230922_PL28/PL28_230922_401_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        'dipping/20230922_PL28/PL28_230922_501_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230922_PL28/PL28_230922_601_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230922_PL28/PL28_230922_701_full_wch0_aniso4_scale4AFS2PL4_toref40_wxy.npy',
        'dipping/20230920_PL58/PL58_230920_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230920_PL58/PL58_230920_101_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230920_PL58/PL58_230920_201_full_wch0_aniso4_scale4AFS2PL4_toref20_wxy.npy',
        'dipping/20230920_PL58/PL58_230920_301_full_wch0_aniso4_scale4AFS2PL4_toref40_wxy.npy',
        'dipping/20230920_PL56/PL56_230920_001_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230920_PL56/PL56_230920_101_full_wch0_aniso4_scale4AFS2PL4_toref4_wxy.npy',
        'dipping/20230920_PL56/PL56_230920_201_full_wch0_aniso4_scale4AFS2PL4_toref5_wxy.npy',
        'dipping/20230916_PL54/PL54_230916_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        'dipping/20230916_PL54/PL54_230916_101_full_wch0_aniso4_scale4AFS2PL4_toref13_wxy.npy',
        'dipping/20230916_PL54/PL54_230916_201_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230916_PL54/PL54_230916_301_full_wch0_aniso4_scale4AFS2PL4_toref19_wxy.npy',
        'dipping/20230916_PL54/PL54_230916_401_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230916_PL54/PL54_230916_501_full_wch0_aniso4_scale4AFS2PL4_toref35_wxy.npy',
        'dipping/20230916_PL48/PL48_230916_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230916_PL48/PL48_230916_101_full_wch0_aniso4_scale4AFS2PL4_toref40_wxy.npy',
        'dipping/20230915_PL52/PL52_230915_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        'dipping/20230915_PL52/PL52_230915_101_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230915_PL52/PL52_230915_201_full_wch0_aniso4_scale4AFS2PL4_toref31_wxy.npy',
        'dipping/20230915_PL52/PL52_230915_301_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230915_PL52/PL52_230915_401_full_wch0_aniso4_scale4AFS2PL4_toref19_wxy.npy']

jumps = np.repeat(31,len(paths)).tolist()

In [5]:
# cleaning up mistakes in cutoffs!

base_path = r'//nasquatch/data/2p/peter/'

paths = ['in_vivo/220408_PK27PL20/PK27PL20_220408_101_full_wch0_aniso4_scale4AFS2PL4_toref57_wxy.npy',
        'slice/20230824_SL1PL60/PL60_230824_001_full_wch1_aniso4_scale4AFS2PL4_toref18_wxy.npy']

jumps = np.repeat(52,len(paths)).tolist()

In [6]:
base_path = r'//nasquatch/data/2p/peter/'

paths = ['dipping/20220921_PL25/PL25_220921_001_full_wch0_aniso4_scale4AFS2PL4_toref19_wxy.npy']

jumps = np.repeat(52,len(paths)).tolist()

In [4]:
# final in vivo movies (up to 2022 7/1)
# final peter/dipping movies (all 2022)
base_path = r'//nasquatch/data/2p/peter/'

paths = ['in_vivo/220701_PK49PL18/PK49PL18_220701_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref76_wxy.npy',
        'in_vivo/220701_PK49PL18/PK49PL18_220701_101_full_wch0_aniso4_scale4AFS2PL4_toref100_wxy.npy',
        'in_vivo/220629_PK46PL25/PK46PL25_220629_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref76_wxy.npy',
        'in_vivo/220629_PK46PL25/PK46PL25_220629_101_full_wch0_aniso4_scale4AFS2PL4_toref20_wxy.npy',
        'in_vivo/220628_PK45PL24/PK45PL24_220628_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/220624_PK45PL13/PK45PL13_220624_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref70_wxy.npy',
        'in_vivo/220624_PK45PL13/PK45PL13_220624_101_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/220622_PK44PL17/PK44PL17_220622_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref76_wxy.npy',
        'in_vivo/220622_PK44PL17/PK44PL17_220622_101_full_wch0_aniso4_scale4AFS2PL4_toref115_wxy.npy',
        'in_vivo/220621_PK24PL13/PK24PL13_220621_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref67_wxy.npy',
        'in_vivo/220621_PK24PL13/PK25PL13_220621_101_full_wch0_aniso4_scale4AFS2PL4_toref10_wxy.npy',
        'in_vivo/220617_PK26PL18/PK26PL18_220617_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref40_wxy.npy',
        'in_vivo/220617_PK26PL18/PK26PL18_220617_101_full_wch0_aniso4_scale4AFS2PL4_toref120_wxy.npy',
        'in_vivo/220616_PK25PL9/PK25PL9_220616_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref76_wxy.npy',
        'in_vivo/220615_PK24PL18/PK24PL18_220615_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref35_wxy.npy',
        'in_vivo/220530_PK25PL9/PK25PL9_220530_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref20_wxy.npy',
        'in_vivo/220530_PK25PL9/PK25PL9_220530_101_full_wch0_aniso4_scale4AFS2PL4_toref80_wxy.npy',
        'in_vivo/220519_PK43PL17/PK43PL17_220519_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref76_wxy.npy',
        'in_vivo/220519_PK43PL17/PK43PL17_220519_101_full_wch0_aniso4_scale4AFS2PL4_toref115_wxy.npy',
        'in_vivo/220518_PK40PL14/PK40PL14_220518_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref50_wxy.npy',
        'in_vivo/220515_PK27PL13/PK27PL13_220515_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref60_wxy.npy',
        'in_vivo/220514_PK26PL6/PK26PL6_220514_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'in_vivo/220514_PK26PL6/PK26PL6_220514_101_full_wch0_aniso4_scale4AFS2PL4_toref60_wxy.npy',
        'in_vivo/220512_PK24PL19/PK24PL19_220512_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/220506_PK26PL16/PK26PL16_220506_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref76_wxy.npy',
        'in_vivo/220504_PK24PL9/PK24PL9_220504_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref80_wxy.npy',
        'in_vivo/220504_PK24PL9/PK24PL9_220504_101_full_wch0_aniso4_scale4AFS2PL4_toref13_wxy.npy',
        'in_vivo/220427_PK27PL20/PK27PL20_220427_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref77_wxy.npy',
        'in_vivo/220427_PK27PL20/PK27PL20_220427_101_full_wch0_aniso4_scale4AFS2PL4_toref80_wxy.npy',
        'in_vivo/220423_PK24PL17/PK24PL17_220423_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref67_wxy.npy',
        'in_vivo/220408_PK27PL20/PK27PL20_220408_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/220408_PK27PL20/PK27PL20_220408_101_full_wch0_aniso4_scale4AFS2PL4_toref57_wxy.npy',
        'in_vivo/220407_PK26PL15/PK26PL15_220407_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref20_wxy.npy',
        'in_vivo/220407_PK26PL15/PK26PL15_220407_101_full_wch0_aniso4_scale4AFS2PL4_toref57_wxy.npy',
        'in_vivo/220406_PK24PL10/PK24PL10_220406_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref40_wxy.npy',
        'in_vivo/220406_PK24PL10/PK24PL10_220406_101_full_wch0_aniso4_scale4AFS2PL4_toref57_wxy.npy',
        'in_vivo/220318_PK24PL14/PK24PL14_220318_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref80_wxy.npy',
        'in_vivo/220318_PK24PL14/PK24PL14_220318_101_full_wch0_aniso4_scale4AFS2PL4_toref62_wxy.npy',
        'in_vivo/220316_PK32PL13/PK32PL13_220316_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref70_wxy.npy',
        'in_vivo/220316_PK32PL13/PK32PL13_220316_101_full_wch0_aniso4_scale4AFS2PL4_toref5_wxy.npy',
        'in_vivo/220315_PK36PL18/PK36PL18_220315_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/220315_PK36PL18/PK36PL18_220315_101_full_wch0_aniso4_scale4AFS2PL4_toref21_wxy.npy',
        'in_vivo/220314_PL14BCH/PL14BCH_220314_001_full_wch0_aniso4_scale4AFS2PL4_toref57_wxy.npy',
        'in_vivo/220311_PK24PL9/PK24PL9_220311_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref62_wxy.npy',
        'in_vivo/220311_PK24PL9/PK24PL9_220311_101_full_wch0_aniso4_scale4AFS2PL4_toref100_wxy.npy',
        'in_vivo/220303_PK26PL21/PK26PL21_220303_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref50_wxy.npy',
        'in_vivo/220303_PK26PL21/PK26PL21_220303_101_full_wch0_aniso4_scale4AFS2PL4_toref91_wxy.npy',
        'in_vivo/220302_PK27PL13/PK27PL13_220302_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref62_wxy.npy',
        'in_vivo/220223_PK26PL19/PK26PL19_220223_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref67_wxy.npy',
        'in_vivo/220217_PK27PL13/PK27PL13_220217_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref58_wxy.npy',
        'in_vivo/220217_PK26PL15/PK26PL15_220217_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref53_wxy.npy',
        'in_vivo/220216_PK25PL8/PK25PL8_220216_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref62_wxy.npy',
        'in_vivo/220216_PK25PL8/PK25PL8_220216_101_full_wch0_aniso4_scale4AFS2PL4_toref45_wxy.npy',
        'dipping/20221129_PL28/PL28_221129_001_full_wch0_aniso4_scale4AFS2PL4_toref40_wxy.npy',
        'dipping/20221129_PL28/PL28_221129_101_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        'dipping/20221118_PL28/PL28_221118_001_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'dipping/20221116_PL44/PL44_221116_001_full_wch0_aniso4_scale4AFS2PL4_toref76_wxy.npy',
        'dipping/20221115_PL42/PL42_221115_001_full_wch0_aniso4_scale4AFS2PL4_toref10_wxy.npy',
        'dipping/20221115_PL42/PL42_221115_101_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'dipping/20221115_PL42/PL42_221115_201_full_wch0_aniso4_scale4AFS2PL4_toref16_wxy.npy',
        'dipping/20221104_PL45/PL45_221104_001_full_wch0_aniso4_scale4AFS2PL4_toref66_wxy.npy',
        'dipping/20221103_PL47/PL47_221103_001_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'dipping/20221103_PL38/PL38_221103_001_full_wch0_aniso4_scale4AFS2PL4_toref76_wxy.npy',
        'dipping/20221102_PL44/PL44_221102_001_full_wch0_aniso4_scale4AFS2PL4_toref19_wxy.npy',
        'dipping/20221102_PL44/PL44_221102_101_full_wch0_aniso4_scale4AFS2PL4_toref115_wxy.npy',
        'dipping/20221101_PL42/PL42_221101_001_full_wch0_aniso4_scale4AFS2PL4_toref19_wxy.npy',
        'dipping/20221101_PL42/PL42_221101_101_full_wch0_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'dipping/20221014_PL36/PL36_221014_001_full_wch0_aniso4_scale4AFS2PL4_toref19_wxy.npy',
        'dipping/20221014_PL36/PL36_221014_101_full_wch0_aniso4_scale4AFS2PL4_toref16_wxy.npy',
        'dipping/20221013_PL33/PL33_221013_001_full_wch0_aniso4_scale4AFS2PL4_toref19_wxy.npy',
        'dipping/20221013_PL33/PL33_221013_101_full_wch0_aniso4_scale4AFS2PL4_toref80_wxy.npy',
        'dipping/20220930_PL34/PL34_220930_001_full_wch0_aniso4_scale4AFS2PL4_toref13_wxy.npy',
        'dipping/20220930_PL34/PL34_220930_101_full_wch0_aniso4_scale4AFS2PL4_toref50_wxy.npy',
        'dipping/20220930_PL34/PL34_220930_101_full_wch0_aniso4_scale4AFS2PL4_toref50_wxy.npy',
        'dipping/20220921_PL25/PL25_220921_101_full_wch0_aniso4_scale4AFS2PL4_toref46_wxy.npy',
        'dipping/20220915_PL29/PL29_220915_001_full_wch0_aniso4_scale4AFS2PL4_toref19_wxy.npy',
        'dipping/20220915_PL29/PL29_220915_101_full_wch0_aniso4_scale4AFS2PL4_toref11_wxy.npy',
        'dipping/20220914_PL21/PL21_220914_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        'dipping/20220914_PL21/PL21_220914_101_full_wch0_aniso4_scale4AFS2PL4_toref115_wxy.npy',
        'dipping/20220907_PL27/PL27_220907_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        'dipping/20220907_PL27/PL27_220907_101_full_wch0_aniso4_scale4AFS2PL4_toref115_wxy.npy',
        'dipping/20220906_PL24/PL24_220906_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        'dipping/20220906_PL24/PL24_220906_101_full_wch0_aniso4_scale4AFS2PL4_toref115_wxy.npy',
        'dipping/20220831_PL31/PL31_220831_001_full_wch0_aniso4_scale4AFS2PL4_toref57_wxy.npy',
        'dipping/20220831_PL27/PL27_220831_001_full_wch0_aniso4_scale4AFS2PL4_toref57_wxy.npy',
        'dipping/20220830_PL32/PL32_220830_001_full_wch0_aniso4_scale4AFS2PL4_toref10_wxy.npy',
        'dipping/20220830_PL24/PL24_220830_001_full_wch0_aniso4_scale4AFS2PL4_toref10_wxy.npy',
        'dipping/20220825_PL29/PL29_220825_001_full_wch0_aniso4_scale4AFS2PL4_toref20_wxy.npy',
        'dipping/20220825_PL18/PL18_220825_001_full_wch0_aniso4_scale4AFS2PL4_toref20_wxy.npy',
        'dipping/20220824_PL33/PL33_220824_001_full_wch0_aniso4_scale4AFS2PL4_toref12_wxy.npy',
        'dipping/20220824_PL13/PL13_220824_001_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'dipping/20220823_PL32/PL32_220823_001_full_wch0_aniso4_scale4AFS2PL4_toref10_wxy.npy',
        'dipping/20220823_PL24/PL24_220823_001_full_wch0_aniso4_scale4AFS2PL4_toref13_wxy.npy',
        'dipping/20220819_PL23/PL23_220819_001_full_wch0_aniso4_scale4AFS2PL4_toref10_wxy.npy',
        'dipping/20220819_PL22/PL22_220819_001_full_wch0_aniso4_scale4AFS2PL4_toref20_wxy.npy',
        'dipping/20220817_PL31/PL31_220817_001_full_wch0_aniso4_scale4AFS2PL4_toref20_wxy.npy',
        'dipping/20220817_PL27/PL27_220817_001_full_wch0_aniso4_scale4AFS2PL4_toref57_wxy.npy',
        'dipping/20220816_PL24/PL24_220816_001_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'dipping/20220812_PL28/PL28_220812_001_full_wch0_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20220812_PL22/PL22_220812_001_full_wch0_aniso4_scale4AFS2PL4_toref55_wxy.npy',
        'dipping/20220811_PL30/PL30_220811_001_full_wch0_aniso4_scale4AFS2PL4_toref40_wxy.npy',
        'dipping/20220811_PL18/PL18_220811_001_full_wch0_aniso4_scale4AFS2PL4_toref48_wxy.npy',
        'dipping/20220810_PL27/PL27_220810_001_full_wch0_aniso4_scale4AFS2PL4_toref8_wxy.npy',
        'dipping/20220810_PL20/PL20_220810_001_full_wch0_aniso4_scale4AFS2PL4_toref10_wxy.npy',
        'dipping/20220809_PL24/PL24_220809_001_full_wch0_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20220809_PL14/PL14_220809_001_full_wch0_aniso4_scale4AFS2PL4_toref10_wxy.npy',
        'dipping/20220729_PL27/PL27_220729_001_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'dipping/20220728_PL24/PL24_220728_001_full_wch0_aniso4_scale4AFS2PL4_toref13_wxy.npy',
        'dipping/20220727_PL14/PL14_220727_001_full_wch0_aniso4_scale4AFS2PL4_toref8_wxy.npy',
        'dipping/20220707_PL18/PL18_220707_001_full_wch0_aniso4_scale4AFS2PL4_toref10_wxy.npy',
        'dipping/20220705_PL9/PL9_220705_001_full_wch0_aniso4_scale4AFS2PL4_toref40_wxy.npy']

jumps = np.repeat(52,len(paths)).tolist()

In [7]:
# next in vivo movies (2022 9/1 - 2023 8/10)
base_path = r'//nasquatch/data/2p/peter/'

paths = ['in_vivo/230810_DG87PL55/DG87PL55_230810_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref94_wxy.npy',
        'in_vivo/230810_DG87PL55/DG87PL55_230810_101_full_wch0_aniso4_scale4AFS2PL4_toref48_wxy.npy',
        'in_vivo/230809_DG86PL57/DG86PL57_230809_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref89_wxy.npy',
        'in_vivo/230809_DG86PL57/DG86PL57_230809_101_full_wch0_aniso4_scale4AFS2PL4_toref48_wxy.npy',
        'in_vivo/230808_DG85PL61/DG85PL61_230808_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref55_wxy.npy',
        'in_vivo/230808_DG85PL61/DG85PL61_230808_101_full_wch0_aniso4_scale4AFS2PL4_toref48_wxy.npy',
        'in_vivo/230807_DG84PL52/DG84PL52_230807_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref73_wxy.npy',
        'in_vivo/230807_DG84PL52/DG84PL52_230807_101_full_wch0_aniso4_scale4AFS2PL4_toref48_wxy.npy',
        'in_vivo/230802_DG48PL58/DG48PL58_230802_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/230802_DG48PL58/DG48PL58_230802_101_full_wch0_aniso4_scale4AFS2PL4_toref57_wxy.npy',
        'in_vivo/230728_DG80PL57/DG80PL57_230728_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref10_wxy.npy',
        'in_vivo/230727_DG79PL59/DG79PL59_230727_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/230726_DG47PL54/DG47PL54_230726_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/230726_DG47PL54/DG47PL54_230726_101_full_wch0_aniso4_scale4AFS2PL4_toref30_wxy.npy',
        'in_vivo/230721_DG46PL32/DG46PL32_230721_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/230721_DG46PL32/DG46PL32_230721_101_full_wch0_aniso4_scale4AFS2PL4_toref57_wxy.npy',
        'in_vivo/230720_PK97PL48/PK97PL48_230720_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref70_wxy.npy',
        'in_vivo/230720_PK97PL48/PK97PL48_230720_101_full_wch0_aniso4_scale4AFS2PL4_toref50_wxy.npy',
        'in_vivo/230714_PK95PL55/PK95PL55_230714_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'in_vivo/230713_PK94PL59/PK94PL59_230713_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/230713_PK94PL59/PK94PL59_230713_101_full_wch0_aniso4_scale4AFS2PL4_toref50_wxy.npy',
        'in_vivo/230712_PK93PL57/PK93PL57_230712_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref35_wxy.npy',
        'in_vivo/230712_PK93PL57/PK93PL57_230712_101_full_wch0_aniso4_scale4AFS2PL4_toref57_wxy.npy',
        'in_vivo/230711_DG44PL47/DG44PL47_230711_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref53_wxy.npy',
        'in_vivo/230711_DG44PL47/DG44PL47_230711_101_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/230711_DG43PL32/DG43PL32_230711_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref29_wxy.npy',
        'in_vivo/230708_DG70PL56/DG70PL56_230708_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/230708_DG70PL56/DG70PL56_230708_101_full_wch0_aniso4_scale4AFS2PL4_toref57_wxy.npy',
        'in_vivo/230707_DG72PL32/DG72PL32_230707_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref40_wxy.npy',
        'in_vivo/230707_DG72PL32/DG72PL32_230707_101_full_wch0_aniso4_scale4AFS2PL4_toref35_wxy.npy',
        'in_vivo/230706_DG71PL54/DG71PL54_230706_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'in_vivo/230706_DG71PL54/DG71PL54_230706_002_wch0_aniso4_scale4AFS2PL4_toref3_wxy.npy',#mouse2??
        'in_vivo/230706_DG71PL54/DG71PL54_230706_101_full_wch0_aniso4_scale4AFS2PL4_toref30_wxy.npy',
        'in_vivo/230617_DG70PL54/DG70PL54_230617_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/230617_DG70PL54/DG70PL54_230617_101_full_wch0_aniso4_scale4AFS2PL4_toref40_wxy.npy',
        'in_vivo/230615_DG69PL32/DG69PL32_230615_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/230615_DG69PL32/DG69PL32_230615_101_full_wch0_aniso4_scale4AFS2PL4_toref54_wxy.npy',
        'in_vivo/230614_DG63PL54/DG63PL54_230614_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref50_wxy.npy',
        'in_vivo/230614_DG63PL54/DG63PL54_230614_101_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/230601_PK85PL53/PK85PL53_230601_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/230601_PK85PL53/PL53_230601_101_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/230531_PK80PL43/PK80PL43_230531_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref20_wxy.npy',
        'in_vivo/230531_PK80PL43/PL43_230531_101_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/230526_PK84PL50/PK84PL50_230526_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref62_wxy.npy',
        'in_vivo/230526_PK84PL50/PL50_230526_101_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/230525_DG42PL39/DG42PL39_230525_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref70_wxy.npy',
        'in_vivo/230525_DG42PL39/PL39_230525_101_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/230519_DG41PL52/DG41PL52_230519_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/230519_DG41PL52/PL52_230519_101_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/221204_PS39PL51/PS39PL51_221204_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref58_wxy.npy',
        'in_vivo/221204_PS39PL51/PS39PL51_221204_101_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/221203_PS45PL50/PS45PL50_221203_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref30_wxy.npy',
        'in_vivo/221203_PS45PL50/PS45PL50_221203_101_full_wch0_aniso4_scale4AFS2PL4_toref115_wxy.npy',
        'in_vivo/221125_PS39PL50/PS39PL50_221125_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref58_wxy.npy',
        'in_vivo/221125_PS39PL50/PS39PL50_221125_101_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/221111_PS28PL41/PS28PL41_221111_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref40_wxy.npy',
        'in_vivo/221111_PS28PL41/PS28PL41_221111_101_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/221109_PS27PL28/PS27PL28_221109_0011_noartifact_wch0_aniso4_scale4AFS2PL4_toref58_wxy.npy',
        'in_vivo/221109_PS27PL28/PS27PL28_221109_001_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/221107_PS23PL42/PS23PL42_221107_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref10_wxy.npy',
        'in_vivo/221020_PS16PL39/PS16PL39_221020_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        'in_vivo/221020_PS16PL39/PS16PL39_221020_101_full_wch0_aniso4_scale4AFS2PL4_toref115_wxy.npy',
        'in_vivo/221018_PS10PL24/PS10PL24_221018_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref31_wxy.npy',
        'in_vivo/221018_PS10PL24/PS10PL24_221018_101_full_wch0_aniso4_scale4AFS2PL4_toref20_wxy.npy',
        'in_vivo/220929_PS11PL37/PS11PL37_220929_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref53_wxy.npy',
        'in_vivo/220928_PK65PL30/PK65PL30_220928_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref53_wxy.npy',
        'in_vivo/220927_PK60PL33/PK60PL33_220927_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref53_wxy.npy',
        'in_vivo/220916_PK66PL28/PK66PL28_220916_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/220916_PK66PL28/PK66PL28_220916_101_full_wch0_aniso4_scale4AFS2PL4_toref115_wxy.npy',
        'in_vivo/220913_PK65PL32/PK65PL32_220913_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref40_wxy.npy',
        'in_vivo/220903_PK63PL34/PK63PL34_220903_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref55_wxy.npy',
        'in_vivo/220903_PK63PL34/PK63PL34_220903_101_full_wch0_aniso4_scale4AFS2PL4_toref115_wxy.npy',
        'in_vivo/220902_PK61PL37/PK61PL37_220902_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/220902_PK61PL37/PPK61PL37_220902_101_full_wch0_aniso4_scale4AFS2PL4_toref1_wxy.npy',
        'in_vivo/220901_PK60PL36/PK60PL36_220901_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref67_wxy.npy',
        'in_vivo/220901_PK60PL36/PK60PL36_220901_101_full_wch0_aniso4_scale4AFS2PL4_toref60_wxy.npy']

jumps = np.repeat(52,len(paths)).tolist()

In [13]:
# second set of 2023 dipping
base_path = r'//nasquatch/data/2p/peter/'

paths = ['dipping/20230825_PL57/PL57_230825_001_full_wch1_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        'dipping/20230825_PL39/PL39_230825_001_full_wch1_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230823_PL61/PL61_230823_001_full_wch1_aniso4_scale4AFS2PL4_toref27_wxy.npy',
        'dipping/20230823_PL48/PL48_230823_001_full_wch1_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230823_PL28/PL28_230823_001_full_wch1_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        'dipping/20230823_PL28/PL28_230823_101_full_wch1_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230823_PL28/PL28_230823_201_full_wch1_aniso4_scale4AFS2PL4_toref46_wxy.npy',
        'dipping/20230821_PL57/PL57_230821_001_full_wch1_aniso4_scale4AFS2PL4_toref22_wxy.npy',
        'dipping/20230819_PL62/PL62_230819_001_full_wch1_aniso4_scale4AFS2PL4_toref20_wxy.npy',
        'dipping/20230819_PL43/PL43_230819_001_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'dipping/20230819_PL43/PL43_230819_101_full_wch1_aniso4_scale4AFS2PL4_toref31_wxy.npy',
        'dipping/20230819_PL43/PL43_230819_201_full_wch1_aniso4_scale4AFS2PL4_toref1_wxy.npy']

jumps = np.repeat(52,len(paths)).tolist()

In [5]:
# 2023 dipping - two sets, one has jumps 31 one 52; use warp to ch1!
# this if first set 9/15 onward
base_path = r'//nasquatch/data/2p/peter/'

paths = ['dipping/20230927_PL58/PL58_230927_001_full_wch1_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230927_PL58/PL58_230927_101_full_wch1_aniso4_scale4AFS2PL4_toref31_wxy.npy',
        'dipping/20230927_PL58/PL58_230927_201_full_wch1_aniso4_scale4AFS2PL4_toref30_wxy.npy',
        'dipping/20230927_PL58/PL58_230927_301_full_wch1_aniso4_scale4AFS2PL4_toref30_wxy.npy',
        'dipping/20230927_PL34/PL34_230927_001_full_wch1_aniso4_scale4AFS2PL4_toref20_wxy.npy',
        'dipping/20230927_PL34/PL34_230927_101_full_wch1_aniso4_scale4AFS2PL4_toref35_wxy.npy',
        'dipping/20230927_PL34/PL34_230927_201_full_wch1_aniso4_scale4AFS2PL4_toref37_wxy.npy',
        'dipping/20230927_PL34/PL34_230927_301_full_wch1_aniso4_scale4AFS2PL4_toref30_wxy.npy',
        'dipping/20230926_PL56/PL56_230926_001_full_wch1_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230926_PL56/PL56_230926_101_full_wch1_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230926_PL56/PL56_230926_201_full_wch1_aniso4_scale4AFS2PL4_toref32_wxy.npy',
        'dipping/20230926_PL56/PL56_230926_301_full_wch1_aniso4_scale4AFS2PL4_toref35_wxy.npy',
        'dipping/20230926_PL56/PL56_230926_401_full_wch1_aniso4_scale4AFS2PL4_toref30_wxy.npy',
        'dipping/20230926_PL56/PL56_230926_501_full_wch1_aniso4_scale4AFS2PL4_toref31_wxy.npy',
        'dipping/20230925_PL58/PL58_230925_001_full_wch1_aniso4_scale4AFS2PL4_toref30_wxy.npy',
        'dipping/20230925_PL58/PL58_230925_101_full_wch1_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230925_PL58/PL58_230925_201_full_wch1_aniso4_scale4AFS2PL4_toref37_wxy.npy',
        'dipping/20230925_PL58/PL58_230925_301_full_wch1_aniso4_scale4AFS2PL4_toref31_wxy.npy',
        'dipping/20230925_PL58/PL58_230925_401_full_wch1_aniso4_scale4AFS2PL4_toref31_wxy.npy',
        'dipping/20230923_PL35/PL35_230923_001_full_wch1_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230923_PL35/PL35_230923_101_full_wch1_aniso4_scale4AFS2PL4_toref37_wxy.npy',
        'dipping/20230923_PL35/PL35_230923_201_full_wch1_aniso4_scale4AFS2PL4_toref37_wxy.npy',
        'dipping/20230923_PL35/PL35_230923_301_full_wch1_aniso4_scale4AFS2PL4_toref26_wxy.npy',
        'dipping/20230923_PL35/PL35_230923_401_full_wch1_aniso4_scale4AFS2PL4_toref36_wxy.npy',
        'dipping/20230923_PL35/PL35_230923_501_full_wch1_aniso4_scale4AFS2PL4_toref31_wxy.npy',
        'dipping/20230922_PL28/PL28_230922_001_full_wch1_aniso4_scale4AFS2PL4_toref35_wxy.npy',
        'dipping/20230922_PL28/PL28_230922_101_full_wch1_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230922_PL28/PL28_230922_201_full_wch1_aniso4_scale4AFS2PL4_toref19_wxy.npy',
        'dipping/20230922_PL28/PL28_230922_301_full_wch1_aniso4_scale4AFS2PL4_toref35_wxy.npy',
        'dipping/20230922_PL28/PL28_230922_401_full_wch1_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        'dipping/20230922_PL28/PL28_230922_501_full_wch1_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230922_PL28/PL28_230922_601_full_wch1_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230922_PL28/PL28_230922_701_full_wch1_aniso4_scale4AFS2PL4_toref40_wxy.npy',
        'dipping/20230920_PL58/PL58_230920_001_full_wch1_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230920_PL58/PL58_230920_101_full_wch1_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230920_PL58/PL58_230920_201_full_wch1_aniso4_scale4AFS2PL4_toref20_wxy.npy',
        'dipping/20230920_PL58/PL58_230920_301_full_wch1_aniso4_scale4AFS2PL4_toref40_wxy.npy',
        'dipping/20230920_PL56/PL56_230920_001_full_wch1_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230920_PL56/PL56_230920_101_full_wch1_aniso4_scale4AFS2PL4_toref4_wxy.npy',
        'dipping/20230920_PL56/PL56_230920_201_full_wch1_aniso4_scale4AFS2PL4_toref5_wxy.npy',
        'dipping/20230916_PL54/PL54_230916_001_full_wch1_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        'dipping/20230916_PL54/PL54_230916_101_full_wch1_aniso4_scale4AFS2PL4_toref13_wxy.npy',
        'dipping/20230916_PL54/PL54_230916_201_full_wch1_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230916_PL54/PL54_230916_301_full_wch1_aniso4_scale4AFS2PL4_toref19_wxy.npy',
        'dipping/20230916_PL54/PL54_230916_401_full_wch1_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230916_PL54/PL54_230916_501_full_wch1_aniso4_scale4AFS2PL4_toref35_wxy.npy',
        'dipping/20230916_PL48/PL48_230916_001_full_wch1_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230916_PL48/PL48_230916_101_full_wch1_aniso4_scale4AFS2PL4_toref40_wxy.npy',
        'dipping/20230915_PL52/PL52_230915_001_full_wch1_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        'dipping/20230915_PL52/PL52_230915_101_full_wch1_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        'dipping/20230915_PL52/PL52_230915_201_full_wch1_aniso4_scale4AFS2PL4_toref31_wxy.npy',
        'dipping/20230915_PL52/PL52_230915_301_full_wch1_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        'dipping/20230915_PL52/PL52_230915_401_full_wch1_aniso4_scale4AFS2PL4_toref19_wxy.npy']

jumps = np.repeat(31,len(paths)).tolist()

In [6]:
base_path = r'//nasquatch/data/2p/peter/'

paths = [#'slice/20230831_SL1PL55/PL55_230831_001_full_wch1_aniso4_scale4AFS2PL4_toref52_wxy.npy',
        #'slice/20230831_SL1PL55/SL1PL55_230831_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy.npy',
        #'slice/20230830_SL2PL60/SL2PL60_230830_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy.npy',
        #'slice/20230830_SL1PL28/PL28_230830_001_full_wch1_aniso4_scale4AFS2PL4_toref52_wxy.npy',
        #'slice/20230830_SL1PL28/SL1PL28_230830_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy.npy',
        #'slice/20230829_SL3PL61/SL3PL61_230829_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy.npy',
        #'slice/20230829_SL2PL39/SL2PL39_230829_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy.npy',
        #'slice/20230829_SL1PL56/PL56_230829_001_full_wch1_aniso4_scale4AFS2PL4_toref52_wxy.npy',
        #'slice/20230829_SL1PL56/SL1PL56_230829_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy.npy',
        #'slice/20230824_SL2PL54/SL2PL54_230824_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref33_wxy.npy',
        #'slice/20230824_SL1PL60/PL60_230824_001_full_wch1_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        #'slice/20230824_SL1PL60/PL60_230824_101_full_wch1_aniso4_scale4AFS2PL4_toref20_wxy.npy',
        #'slice/20230824_SL1PL60/SL1PL60_230824_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref22_wxy.npy',
        #'slice/20230822_SL3PL58/SL3PL58_230822_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy.npy',
        #'slice/20230822_SL2PL52/PL52_230822_001_full_wch1_aniso4_scale4AFS2PL4_toref52_wxy.npy',
        #'slice/20230822_SL2PL52/SL2PL52_230822_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy.npy',
        #'slice/20230822_SL1PL32/SL1PL32_230822_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref28_wxy.npy',
        #'slice/20230817_SL2PL60/SL2PL60_230817_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref27_wxy.npy', #from here onward, definitely stop ch1 since no tdTomato!
        #'slice/20230817_SL1PL55/PL55_230817_001_full_wch0_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        #'slice/20230817_SL1PL55/PL55_230817_101_full_wch0_aniso4_scale4AFS2PL4_toref39_wxy.npy',
        #'slice/20230817_SL1PL55/SL1PL55_230817_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref27_wxy.npy',
        #'slice/20230801_SL2PL34/SL2PL34_230801_001_noartifact_wch0_aniso4_scale4.4AFS2PL4_toref27_wxy.npy', 
        #'slice/20230801_SL1PL35/PL35_230801_101_full_wch0_aniso4_scale4AFS2PL4_toref76_wxy.npy',
        #'slice/20230801_SL1PL35/SL1PL35_230801_001_noartifact_wch0_aniso4_scale4.4AFS2PL4_toref27_wxy.npy',
        #'slice/20230727_SL2PL32/PL32_230727_101_full_wch0_aniso4_scale4AFS2PL4_toref39_wxy.npy',
        #'slice/20230727_SL2PL32/SL2PL32_230727_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref25_wxy.npy',
        #'slice/20230727_SL1PL52/SL1PL52_230727_001_noartifact_wch0_aniso4_scale4AFS2PL4_toref18_wxy.npy', #done up to here, then error
        #'slice/20230616_SL2PL39/SL2PL39_230616_001_wch0_aniso4_scale4.4AFS2PL4_toref18_wxy.npy', #THIS FILE HAS A PROBLEM - header?
        #'slice/20230616_SL1PL28/PL28_230616_001_full_wch0_aniso4_scale4.4AFS2PL4_toref11_wxy.npy',
        #'slice/20230616_SL1PL28/SL1PL28_230616_001_wch0_aniso4_scale4.4AFS2PL4_toref18_wxy.npy', #AGAIN A PROBLEM HERE - header?
        #'slice/20230608_SL2PL45/SL2PL45_230608_001_noartifact_wch0_aniso4_scale4.4AFS2PL4_toref26_wxy.npy',
        #'slice/20230608_SL1PL37/SL1PL37_230608_001_noartifact_wch0_aniso4_scale4.4AFS2PL4_toref18_wxy.npy',
        #'slice/20230607_SL2PL42/SL2PL42_230607_001_noartifact_wch0_aniso4_scale4.4AFS2PL4_toref25_wxy.npy',
        #'slice/20230607_SL1PL47/PL47_230607_001_full_wch0_aniso4_scale4.4AFS2PL4_toref27_wxy.npy',
        #'slice/20230607_SL1PL47/SL1PL47_230607_001_wch0_aniso4_scale4.4AFS2PL4_toref15_wxy.npy',
        'slice/20230606_SL1PL35/PL35_230606_001_wch0_aniso4_scale4.4AFS2PL4_toref9_wxy.npy',
        'slice/20230606_SL1PL35/SL1PL35_230606_001_wch0_aniso4_scale4.4AFS2PL4_toref18_wxy.npy']
        #'slice/20230511_DA2PL45/xx0DA2PL45_230511_001_wch0_aniso4_scale4AFS2PL4_toref18_wxy.npy',
        #'slice/20230511_DA1PL37/PL37_230511_101_full_wch0_aniso4_scale4AFS2PL4_toref7_wxy.npy',
        #'slice/20230511_DA1PL37/xx0DA1PL37_230511_001_wch0_aniso4_scale4AFS2PL4_toref15_wxy.npy',
        #'slice/20230509_DA2PL55/DA2PL55_230509_001_wch0_aniso4_scale4AFS2PL4_toref16_wxy.npy',
        #'slice/20230509_DA1PL53/DA1PL53_230509_001_wch0_aniso4_scale4AFS2PL4_toref10_wxy.npy',
        #'slice/20230509_DA1PL53/PL53_230509_101_full_wch0_aniso4_scale4.4AFS2PL4_toref8_wxy.npy']

jumps = [52, 52]
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
        #52,
       # 52]

In [17]:
base_path = r'//nasquatch/data/2p/peter/'

paths = ['in_vivo/231107_DG44PL58/DG44PL58_231107_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref85_wxy.npy',
        'in_vivo/231107_DG44PL58/DG44PL58_231107_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/231106_DG52PL68/DG52PL68_231106_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref85_wxy.npy',
        'in_vivo/231106_DG52PL68/DG52PL68_231106_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/231105_DG39PL32/DG39PL32_231105_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref85_wxy.npy',
        'in_vivo/231105_DG39PL32/DG39PL32_231105_201_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/231105_DG43PL32/DG43PL32_231105_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref19_wxy.npy',
        'in_vivo/231031_DG53PL28/DG53PL28_231031_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref85_wxy.npy',
        'in_vivo/231031_DG53PL28/DG53PL28_231031_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/231030_DG60PL67/DG60PL67_231030_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref49_wxy.npy',
        'in_vivo/231030_DG60PL67/DG60PL67_231030_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/231029_DG58PL64/DG58PL64_231029_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref10_wxy.npy',
        'in_vivo/231027_DG57PL69/DG57PL69_231027_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref85_wxy.npy',
        'in_vivo/231025_DG56PL62/DG56PL62_231025_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref85_wxy.npy',
        'in_vivo/231025_DG56PL62/DG56PL62_231025_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/231024_DG54PL61/DG54PL61_231024_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/231024_DG54PL61/DG54PL61_231024_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/231019_DG53PL68/DG53PL68_231019_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref85_wxy.npy',
        'in_vivo/231018_DG52PL66/DG52PL66_231018_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/231018_DG52PL66/DG52PL66_231018_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/231017_DG49PL70/DG49PL70_231017_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/230909_DG88PL58/DG88PL58_230909_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref71_wxy.npy',
        'in_vivo/230909_DG88PL58/DG88PL58_230909_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/230908_PK109PL52/PK109PL52_230908_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref65_wxy.npy',
        'in_vivo/230908_PK109PL52/PK109PL52_230908_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/230907_PK108PL54/PK108PL54_230907_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref80_wxy.npy',
        'in_vivo/230906_PS57PL56/PS57PL56_230906_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref60_wxy.npy',
        'in_vivo/230906_PS57PL56/PS57PL56_230906_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/230905_PS56PL58/PS56PL58_230905_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref89_wxy.npy',
        'in_vivo/230905_PS56PL58/PS56PL58_230905_101_full_wch1_aniso4_scale4AFS2PL4_toref34_wxy.npy',
        'in_vivo/230904_PS54PL60/PS54PL60_230904_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref89_wxy.npy',
        'in_vivo/230904_PS54PL60/PS54PL60_230904_101_full_wch1_aniso4_scale4AFS2PL4_toref38_wxy.npy',
        'in_vivo/230902_DG90PL61/DG90PL61_230902_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref75_wxy.npy',
        'in_vivo/230902_DG90PL61/DG90PL61_230902_101_full_wch1_aniso4_scale4AFS2PL4_toref38_wxy.npy',
        'in_vivo/230901_DG89PL32/DG89PL32_230901_001_noartifact_wch1_aniso4_scale4AFS2PL4_toref30_wxy.npy',
        'in_vivo/230901_DG89PL32/DG89PL32_230901_101_full_wch1_aniso4_scale4AFS2PL4_toref38_wxy.npy']

jumps = [52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        31,
        52,
        31,
        52,
        52,
        31,
        52,
        52,
        52,
        52,
        52,
        52,
        52,
        52]

In [8]:
read_path = Path(f'{base_path}{paths[1]}')
print(str(read_path))
dat = np.load(read_path,mmap_mode='r+')
#dat = np.load(read_path)
print(dat.shape)
dat = dat.transpose([4,3,0,1,2])
print(dat.shape)
jump = jumps[0]
i = 1
stack = np.array(dat[i-1:i+jump-1,:,:,:,:])
stack.shape

\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\DG61PL62_231108_101_full_wch0_aniso4_scale4AFS2PL4_toref34_wxy.npy
(2, 450, 510, 30, 3588)
(3588, 30, 2, 450, 510)


(52, 30, 2, 450, 510)

In [6]:
loaded_stack_f = apply_func(stack, 'mean')
loaded_stack_f.shape

(30, 2, 450, 510)

In [7]:
'001_full' in str(read_path)

False

In [9]:
from tqdm.auto import trange
from collections import defaultdict

funs = ['mean'] # which function to use | mean | max | median <-- can be a list also
chs = [0] #1 is red, 0 is green

for ch in chs:
    for p in trange(0,len(paths)):
        read_path = Path(f'{base_path}{paths[p]}')
        #if read_path.stem[-8:] == '001_full': #fix all this for warped outputs
        #    save_folder = read_path.parent / f'wells001/segtrack/tiffs/ch{ch}'
        #elif read_path.stem[-8:] == '100_full':
        #    save_folder = read_path.parent / f'wells100/segtrack/tiffs/ch{ch}'
        #elif read_path.stem[-8:] == '101_full':
        #    save_folder = read_path.parent / f'wells101/segtrack/tiffs/ch{ch}'
        #elif read_path.stem[-8:] == '201_full':
        #    save_folder = read_path.parent / f'wells201/segtrack/tiffs/ch{ch}'
        #elif read_path.stem[-8:] == '301_full':
        #    save_folder = read_path.parent / f'wells301/segtrack/tiffs/ch{ch}'
        #elif read_path.stem[-8:] == '401_full':
        #    save_folder = read_path.parent / f'wells401/segtrack/tiffs/ch{ch}'
        #elif read_path.stem[-8:] == '501_full':
        #    save_folder = read_path.parent / f'wells501/segtrack/tiffs/ch{ch}'
        #elif read_path.stem[-8:] == '601_full':
        #    save_folder = read_path.parent / f'wells601/segtrack/tiffs/ch{ch}'
        #elif read_path.stem[-8:] == '701_full':
        #    save_folder = read_path.parent / f'wells701/segtrack/tiffs/ch{ch}'
        #elif read_path.stem[-3:] == '002':
        #    save_folder = read_path.parent / f'mouse2/segtrack/tiffs/ch{ch}'
        #else:
        #    save_folder = read_path.parent / f'wells/segtrack/tiffs/ch{ch}' #keep an eye on these directories!
        if '001_full' in read_path.stem:
            save_folder = read_path.parent / f'wells001/segtrack/tiffs/ch{ch}'
        elif '101_full' in read_path.stem:
            save_folder = read_path.parent / f'wells101/segtrack/tiffs/ch{ch}'
        elif '201_full' in read_path.stem:
            save_folder = read_path.parent / f'wells201/segtrack/tiffs/ch{ch}'
        elif '301_full' in read_path.stem:
            save_folder = read_path.parent / f'wells301/segtrack/tiffs/ch{ch}'
        elif '401_full' in read_path.stem:
            save_folder = read_path.parent / f'wells401/segtrack/tiffs/ch{ch}'
        elif '501_full' in read_path.stem:
            save_folder = read_path.parent / f'wells501/segtrack/tiffs/ch{ch}'
        elif '601_full' in read_path.stem:
            save_folder = read_path.parent / f'wells601/segtrack/tiffs/ch{ch}'
        elif '701_full' in read_path.stem:
            save_folder = read_path.parent / f'wells701/segtrack/tiffs/ch{ch}'
        elif '\PL35_2306' in str(read_path):
            save_folder = read_path.parent / f'washes/segtrack/tiffs/ch{ch}'
        elif '002' in str(read_path):
            save_folder = read_path.parent / f'mouse2/segtrack/tiffs/ch{ch}'
        else:
            save_folder = read_path.parent / f'mouse/segtrack/tiffs/ch{ch}'
        stack_n = 0
        dat = np.load(read_path,mmap_mode='r+')
        #dat = np.load(read_path)
        dat = dat.transpose([4,3,0,1,2])
        r_st = 1
        r_en = int(dat.shape[0])
        jump = jumps[p]
        print(dat.shape, list(range(r_st, r_en, jump)))
        for i in trange(r_st, r_en, jump):
            print(i-1,i+jump-1)
            if (dat.shape[0]-i+1)<jump-1: #excess frames to remove
                print("Skip, !!")
                continue
            stack = np.array(dat[i-1:i+jump-1,:,ch,:,:])
            stack_n +=1

            for fun in funs:
                #print('C')
                loaded_stack_f = apply_func(stack, fun)

                funcs_path = fr'{jumps[p]}_{fun}_warped_ch0'.replace(' ','').replace(':','').replace("'",'')
                full_save_path = Path(fr'{save_folder}/{funcs_path}/')
                full_save_path.mkdir(parents=True, exist_ok=True )
                os.makedirs(full_save_path,exist_ok=True,)
                print(full_save_path / f'image{stack_n:03}.tiff')
                tif_imwrite(full_save_path / f'image{stack_n:03}.tiff', loaded_stack_f)
        
        

  0%|          | 0/2 [00:00<?, ?it/s]

(8819, 30, 2, 450, 510) [1, 53, 105, 157, 209, 261, 313, 365, 417, 469, 521, 573, 625, 677, 729, 781, 833, 885, 937, 989, 1041, 1093, 1145, 1197, 1249, 1301, 1353, 1405, 1457, 1509, 1561, 1613, 1665, 1717, 1769, 1821, 1873, 1925, 1977, 2029, 2081, 2133, 2185, 2237, 2289, 2341, 2393, 2445, 2497, 2549, 2601, 2653, 2705, 2757, 2809, 2861, 2913, 2965, 3017, 3069, 3121, 3173, 3225, 3277, 3329, 3381, 3433, 3485, 3537, 3589, 3641, 3693, 3745, 3797, 3849, 3901, 3953, 4005, 4057, 4109, 4161, 4213, 4265, 4317, 4369, 4421, 4473, 4525, 4577, 4629, 4681, 4733, 4785, 4837, 4889, 4941, 4993, 5045, 5097, 5149, 5201, 5253, 5305, 5357, 5409, 5461, 5513, 5565, 5617, 5669, 5721, 5773, 5825, 5877, 5929, 5981, 6033, 6085, 6137, 6189, 6241, 6293, 6345, 6397, 6449, 6501, 6553, 6605, 6657, 6709, 6761, 6813, 6865, 6917, 6969, 7021, 7073, 7125, 7177, 7229, 7281, 7333, 7385, 7437, 7489, 7541, 7593, 7645, 7697, 7749, 7801, 7853, 7905, 7957, 8009, 8061, 8113, 8165, 8217, 8269, 8321, 8373, 8425, 8477, 8529, 8581, 86

  0%|          | 0/170 [00:00<?, ?it/s]

0 52
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image001.tiff
52 104
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image002.tiff
104 156
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image003.tiff
156 208
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image004.tiff
208 260
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image005.tiff
260 312
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image006.tiff
312 364
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image007.tiff
364 416
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image008.tiff
416 468
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tif

3640 3692
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image071.tiff
3692 3744
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image072.tiff
3744 3796
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image073.tiff
3796 3848
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image074.tiff
3848 3900
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image075.tiff
3900 3952
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image076.tiff
3952 4004
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image077.tiff
4004 4056
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image078.tiff
4056 4108
\\nasquatch\data\2p\peter\in_vivo\231108_DG61P

7280 7332
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image141.tiff
7332 7384
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image142.tiff
7384 7436
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image143.tiff
7436 7488
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image144.tiff
7488 7540
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image145.tiff
7540 7592
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image146.tiff
7592 7644
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image147.tiff
7644 7696
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\mouse\segtrack\tiffs\ch0\52_mean_warped_ch0\image148.tiff
7696 7748
\\nasquatch\data\2p\peter\in_vivo\231108_DG61P

  0%|          | 0/69 [00:00<?, ?it/s]

0 52
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\wells101\segtrack\tiffs\ch0\52_mean_warped_ch0\image001.tiff
52 104
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\wells101\segtrack\tiffs\ch0\52_mean_warped_ch0\image002.tiff
104 156
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\wells101\segtrack\tiffs\ch0\52_mean_warped_ch0\image003.tiff
156 208
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\wells101\segtrack\tiffs\ch0\52_mean_warped_ch0\image004.tiff
208 260
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\wells101\segtrack\tiffs\ch0\52_mean_warped_ch0\image005.tiff
260 312
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\wells101\segtrack\tiffs\ch0\52_mean_warped_ch0\image006.tiff
312 364
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\wells101\segtrack\tiffs\ch0\52_mean_warped_ch0\image007.tiff
364 416
\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\wells101\segtrack\tiffs\ch0\52_mean_warped_ch0\image008.tiff
416 468
\\nasquatch\data\2p\peter\in_vivo\231108_DG6

\\nasquatch\data\2p\peter\in_vivo\231108_DG61PL62\wells101\segtrack\tiffs\ch0\52_mean_warped_ch0\image069.tiff
